In [0]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [0]:
from google.colab import files
uploaded = files.upload()

Saving agr_en_dev.csv to agr_en_dev.csv
Saving agr_en_fb_gold.csv to agr_en_fb_gold.csv
Saving agr_en_train.csv to agr_en_train.csv
Saving agr_en_tw_gold.csv to agr_en_tw_gold.csv


In [0]:
''' LogisticRegression(),
    KNeighborsClassifier(15),
    SVC(kernel ='linear'),
    AdaBoostClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=200),
    SGDClassifier(max_iter=1000, tol=1e-3),
    GaussianNB(var_smoothing=1e-01),'''

" LogisticRegression(),\n    KNeighborsClassifier(15),\n    SVC(kernel ='linear'),\n    AdaBoostClassifier(),\n    DecisionTreeClassifier(),\n    RandomForestClassifier(n_estimators=200),\n    SGDClassifier(max_iter=1000, tol=1e-3),\n    GaussianNB(var_smoothing=1e-01),"

In [0]:
Classifiers = [
    #LogisticRegression(C=0.000000001,solver='liblinear',max_iter=200),
    LogisticRegression(solver='newton-cg')]
    #KNeighborsClassifier(15),
    #SVC(kernel ='linear'),
    #AdaBoostClassifier(),
    #DecisionTreeClassifier(),
    #RandomForestClassifier(n_estimators=200),
    #SGDClassifier(max_iter=1000, tol=1e-3)]
    #GaussianNB(var_smoothing=1e-01),
    #VotingClassifier(estimators=[('lr',LogisticRegression(solver='lbfgs', multi_class='multinomial',random_state=1,max_iter=200)),('rf', RandomForestClassifier(n_estimators=50, random_state=1)), 
    #                             ('gnb', GaussianNB(var_smoothing=1e-01))], voting='hard'),
    #VotingClassifier(estimators=[('dt', DecisionTreeClassifier()), ('knn', KNeighborsClassifier(15)), ('svc', SVC(gamma='scale', kernel ='linear', probability=True))],voting='soft')]

In [0]:
data=pd.read_csv("agr_en_train.csv")

In [0]:
data.columns = ['Id','Comment', 'Aggression']

In [0]:
import re
'''import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')'''

"import nltk\nfrom nltk.corpus import stopwords\nnltk.download('stopwords')"

In [0]:
def tweet_to_words(raw_tweet):
    letters_only = re.sub("[^a-zA-Z]", " ",raw_tweet) 
    words = letters_only.lower().split()
    return( " ".join(words)) 

In [0]:
#data['Comment']=data['Comment'].apply(lambda x: tweet_to_words(x))

In [0]:
df=pd.DataFrame([data['Comment'],data['Aggression']]).T

In [0]:
data_test=pd.read_csv("agr_en_dev.csv")
data_test.columns = ['Id','Comment', 'Aggression']
df1=pd.DataFrame([data_test['Comment'],data_test['Aggression']]).T

In [0]:
frames = [df["Comment"], df1["Comment"]]
result = pd.DataFrame(pd.concat(frames))
result.head()

,Comment
0,"Most of Private Banks ATM's Like HDFC, ICICI e..."
1,"Now question is, Pakistan will adhere to this?"
2,Pakistan is comprised of fake muslims who does...
3,"??we r against cow slaughter,so of course it w..."
4,Wondering why Educated Ambassador is strugglin...


In [0]:
result.shape

(14999, 1)

In [0]:
feature_extraction = TfidfVectorizer()
data_train = feature_extraction.fit_transform(result["Comment"])
#data_train.shape
data_train_1=data_train[0:11999,:]
data_train_2=df['Aggression']
data_train_1.shape

(11999, 24373)

In [0]:
data_test_1=data_train[11999:,:]
data_test_2=df1["Aggression"]
data_test_1.shape

(3000, 24373)

In [0]:
dense_features=data_train_1.toarray()
dense_test_1= data_test_1.toarray()
Accuracy=[]
Model=[]
for classifier in Classifiers:
    try:
        fit = classifier.fit(data_train_1,data_train_2)
        pred = fit.predict(data_test_1)
    except Exception:
        fit = classifier.fit(dense_features,data_train_2)
        pred = fit.predict(dense_test_1)
    class_report = classification_report(data_test_2,pred)
    print(class_report)
    conf=confusion_matrix(data_test_2, pred)
    print(conf)
    accuracy = accuracy_score(pred,data_test_2)
    Accuracy.append(accuracy)
    Model.append(classifier.__class__.__name__)
    print('Accuracy of '+classifier.__class__.__name__+'is '+str(accuracy))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

         CAG       0.49      0.52      0.51      1056
         NAG       0.61      0.75      0.67      1233
         OAG       0.62      0.33      0.43       711

   micro avg       0.57      0.57      0.57      3000
   macro avg       0.57      0.53      0.53      3000
weighted avg       0.57      0.57      0.55      3000

[[546 413  97]
 [265 920  48]
 [293 185 233]]
Accuracy of LogisticRegressionis 0.5663333333333334


FaceBook Test

In [0]:
data_test=pd.read_csv("agr_en_fb_gold.csv")
data_test.columns = ['Id','Comment', 'Aggression']
df2=pd.DataFrame([data_test['Comment'],data_test['Aggression']]).T

In [0]:
frames = [df["Comment"], df2["Comment"]]
result1 = pd.DataFrame(pd.concat(frames))
#result.head()

In [0]:
feature_extraction = TfidfVectorizer()
data_train = feature_extraction.fit_transform(result1["Comment"])
#data_train.shape
data_train_1=data_train[0:11999,:]
data_train_2=df['Aggression']
data_train_1.shape

In [0]:
data_test_1=data_train[11999:,:]
data_test_2=df2["Aggression"]
data_test_1.shape

In [0]:
dense_features=data_train_1.toarray()
dense_test_1= data_test_1.toarray()
Accuracy=[]
Model=[]
for classifier in Classifiers:
    try:
        fit = classifier.fit(data_train_1,data_train_2)
        pred = fit.predict(data_test_1)
    except Exception:
        fit = classifier.fit(dense_features,data_train_2)
        pred = fit.predict(dense_test_1)
    class_report = classification_report(data_test_2,pred,digits=4)
    print(class_report)
    conf=confusion_matrix(data_test_2, pred)
    print(conf)
    accuracy = accuracy_score(pred,data_test_2)
    Accuracy.append(accuracy)
    Model.append(classifier.__class__.__name__)
    print('Accuracy of '+classifier.__class__.__name__+'is '+str(accuracy))

Twitter Test

In [0]:
data_test=pd.read_csv("agr_en_tw_gold.csv")
data_test.columns = ['Id','Comment', 'Aggression']
df3=pd.DataFrame([data_test['Comment'],data_test['Aggression']]).T

In [0]:
frames = [df["Comment"], df3["Comment"]]
result2 = pd.DataFrame(pd.concat(frames))
#result.head()

In [0]:
feature_extraction = TfidfVectorizer()
data_train = feature_extraction.fit_transform(result2["Comment"])
#data_train.shape
data_train_1=data_train[0:11999,:]
data_train_2=df['Aggression']
data_train_1.shape

In [0]:
data_test_1=data_train[11999:,:]
data_test_2=df3["Aggression"]
data_test_1.shape

In [0]:
dense_features=data_train_1.toarray()
dense_test_1= data_test_1.toarray()
Accuracy=[]
Model=[]
for classifier in Classifiers:
    try:
        fit = classifier.fit(data_train_1,data_train_2)
        pred = fit.predict(data_test_1)
    except Exception:
        fit = classifier.fit(dense_features,data_train_2)
        pred = fit.predict(dense_test_1)
    class_report = classification_report(data_test_2,pred)
    print(class_report)
    conf=confusion_matrix(data_test_2, pred)
    print(conf)
    accuracy = accuracy_score(pred,data_test_2)
    Accuracy.append(accuracy)
    Model.append(classifier.__class__.__name__)
    print('Accuracy of '+classifier.__class__.__name__+'is '+str(accuracy))

In [0]:
import numpy as np

***Fast Text***

In [0]:
from gensim.models import FastText

In [0]:
type(data['Comment'])

In [0]:
#creating the modelvector of the specified size of 100
tr_model=FastText(data['Comment'],alpha=0.025,window=5,min_count=1,sg=1,min_n=3)
#seperating each word from the text
tr_word=tr_model.wv.vocab
#using dictionary to store those words and to access them later using Key
tr_dict=dict(zip(list(tr_word),tr_model[tr_word]))
Tr_Features = list(map(lambda y:list(np.sum(list(map(lambda x:list(tr_dict[x]),y)),axis=0)),data['Comment']))
